In [20]:
#######
# IMPORT PACKAGES
#######

import praw
import pandas as pd


# Acessing the reddit api


r = praw.Reddit(client_id="E4TBAGuHsqZbFXXiFgL6YQ",#my client id
                     client_secret="09YZIOa3o46cnK3WbqFTCkgepNPbXg",  #your client secret
                     user_agent="my user agent", #user agent name
                     username = "",     # your reddit username
                     password = "")     # your reddit password

def getSubComments(comment, allComments, verbose=True):
  allComments.append(comment)
  if not hasattr(comment, "replies"):
    replies = comment.comments()
    if verbose: print("fetching (" + str(len(allComments)) + " comments fetched total)")
  else:
    replies = comment.replies
  for child in replies:
    getSubComments(child, allComments, verbose=verbose)


def getAll(r, submissionId, verbose=True):
  submission = r.submission(submissionId)
  submission.comments.replace_more(limit=0) # flatten tree
  comments = submission.comments.list() # all comments
  commentsList = []
  for comment in comments:
    getSubComments(comment, commentsList, verbose=verbose)
  return commentsList



res = getAll(r, "665cyg")

In [21]:
data = []
for comment in res:
    data.append([comment.id, comment.body, comment.parent_id])
        
df = pd.DataFrame(data, columns=['id', 'body', 'parent_id'])
df.head()

,id,body,parent_id
0,dgglnne,"Just never was brought up to be a vegan, why a...",t3_665cyg
1,dgh8cm9,"Wait, why do you specify hard cheese? Is soft ...",t1_dgglnne
2,dghbwjq,"Well, I have personal tastes when it comes to ...",t1_dgh8cm9
3,dghcq45,Interesting! I'm not actually familiar with th...,t1_dghbwjq
4,dghcxhb,They're mostly a British type especially Wensl...,t1_dghcq45


In [22]:
df['TopLevel'] = df['parent_id'].str.contains('t1_')
df.head(100)

,id,body,parent_id,TopLevel
0,dgglnne,"Just never was brought up to be a vegan, why a...",t3_665cyg,False
1,dgh8cm9,"Wait, why do you specify hard cheese? Is soft ...",t1_dgglnne,True
2,dghbwjq,"Well, I have personal tastes when it comes to ...",t1_dgh8cm9,True
3,dghcq45,Interesting! I'm not actually familiar with th...,t1_dghbwjq,True
4,dghcxhb,They're mostly a British type especially Wensl...,t1_dghcq45,True
...,...,...,...,...
95,dghfcs9,"agreed. support local, humane farming to show...",t1_dgh7e7m,True
96,dgh6gck,because no one has adequately explained why it...,t3_665cyg,False
97,dgh6tjj,"Well, factory farming is horrible. Its horribl...",t1_dgh6gck,True
98,dgh6wbo,that has nothing to do with eating vegan. the...,t1_dgh6tjj,True


In [23]:
df.to_csv('WhyArentYouVegan.csv')